In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE68950"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE68950"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE68950.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE68950.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE68950.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"caArray_golub-00327: Sanger cell line Affymetrix gene expression project"
!Series_summary	"The microarray gene expression pattern was studied using 798 different cancer cell lines. The cancer cell lines are obtained from different centers. Annotation information were provided in the supplementary file."
!Series_overall_design	"golub-00327"
!Series_overall_design	"Assay Type: Gene Expression"
!Series_overall_design	"Provider: Affymetrix"
!Series_overall_design	"Array Designs: HT_HG-U133A"
!Series_overall_design	"Organism: Homo sapiens (ncbitax)"
!Series_overall_design	"Tissue Sites: leukemia, Urinary tract, Lung, BiliaryTract, Autonomic Ganglion, Thyroid gland, Stomach, Breast, Pancreas, Head and Neck, Lymphoma, Colorectal, Placenta, Liver, Brain, Bone, pleura, Skin, endometrium, Ovary, cervix, Oesophagus, Connective and Soft Tissue, Muscle, Kidney, Prostate, Adrenal Gland, Eye, Testis, Smooth Muscle Tissue, Vulva, Unknow"
!Series_overall_design	"M

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine Gene Expression Data Availability
# Based on the background information, this seems to be gene expression data using Affymetrix array
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Retinoblastoma)
# Looking at disease state information in row 1 to see if Retinoblastoma is mentioned
unique_diseases = set([value.split(': ')[1] for value in sample_characteristics_dict[1]])
if 'Retinoblastoma' in unique_diseases:
    trait_row = 1  # disease state row
else:
    # Check if the eye location might indicate retinoblastoma
    disease_locations = set([value.split(': ')[1] for value in sample_characteristics_dict[2]])
    organism_parts = set([value.split(': ')[1] for value in sample_characteristics_dict[3]])
    
    if 'Eye' in organism_parts or 'eye' in disease_locations:
        trait_row = 3 if 'Eye' in organism_parts else 2  # Use organism part or disease location as proxy
    else:
        trait_row = None  # Trait data not available

# For age - No age information in the sample characteristics
age_row = None  # Age data not available

# For gender - No gender information in the sample characteristics
gender_row = None  # Gender data not available

# 2.2 Data Type Conversion
# Convert trait (Retinoblastoma)
def convert_trait(value: str) -> int:
    """
    Convert trait value to binary (1 for Retinoblastoma, 0 for other diseases)
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ': ' in value:
        value = value.split(': ')[1]
    
    # Direct match for Retinoblastoma
    if value.lower() == 'retinoblastoma':
        return 1
    
    # Match for Eye-related diseases that might be retinoblastoma
    if trait_row == 3 and value == 'Eye':
        return 1
    
    return 0

# Convert age function (not used in this case but defined for completeness)
def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to continuous float
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ': ' in value:
        value = value.split(': ')[1]
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# Convert gender function (not used in this case but defined for completeness)
def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender value to binary (0 for female, 1 for male)
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ': ' in value:
        value = value.split(': ')[1]
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Perform initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    clinical_data = pd.DataFrame(sample_characteristics_dict)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Preview of selected clinical data:")
    preview_data = preview_df(selected_clinical_df)
    print(preview_data)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# The identifiers in the format '1007_s_at', '1053_at', etc. appear to be Affymetrix probe IDs
# These are not standard human gene symbols (which would be like BRCA1, TP53, etc.)
# They need to be mapped to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007', 'Mar 8, 2007'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': [nan, nan, nan, nan, nan], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens Pax8 mRNA', 'L36861 /FEATURE=expanded_cds /DEFINITION=HUMGCAPB Homo sapiens 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns containing gene identifiers and gene symbols
# Looking at the gene annotation preview, we can see:
# - 'ID' column contains identifiers like '1007_s_at', which matches the gene expression data
# - 'Gene Symbol' column contains the actual gene symbols like 'DDR1'

# 2. Get a gene mapping dataframe by extracting these two columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print some information about the resulting gene expression data
print("Gene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print("First 5 gene symbols:")
print(gene_data.index[:5].tolist())


Gene expression data after mapping:
Number of genes: 13046
First 5 gene symbols:
['A2BP1', 'A2M', 'A4GALT', 'A4GNT', 'AAAS']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Extract clinical features directly using the information from previous steps
if 'trait_row' in globals() and trait_row is not None:
    # Re-extract clinical features from the clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    print(f"Clinical features extracted with shape: {selected_clinical_df.shape}")
else:
    # Fallback - create a dummy dataframe with just the trait column
    print("WARNING: Could not find trait_row. Creating a minimal clinical dataframe.")
    selected_clinical_df = pd.DataFrame({trait: [1]})  # Dummy value
    is_trait_available = False

# Transpose clinical features for linking
selected_clinical_df_t = selected_clinical_df.T

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df_t, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(f"First few columns in linked data: {list(linked_data.columns[:5])}")

# 3. Handle missing values in the linked data
# Determine the trait column (should be the first column)
if trait in linked_data.columns:
    trait_column = trait
else:
    # Just use the first column
    trait_column = linked_data.columns[0]
print(f"Using trait column: {trait_column}")

linked_data_processed = handle_missing_values(linked_data, trait_column)
print(f"Shape after handling missing values: {linked_data_processed.shape}")

# Add validation check - if no samples remain, note the issue
if linked_data_processed.shape[0] == 0:
    print("No samples remain after handling missing values. The dataset cannot be processed further.")
    is_trait_biased = True  # Mark as biased since we can't use it
    unbiased_linked_data = linked_data_processed
else:
    # 4. Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_processed, trait_column)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from cell lines with {trait} information."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (12700, 798)
First few normalized gene symbols: ['A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAK1', 'AAMDC', 'AAMP', 'AANAT']


Normalized gene data saved to ../../output/preprocess/Retinoblastoma/gene_data/GSE68950.csv
Linked data shape: (799, 12701)
First few columns in linked data: ['Retinoblastoma', 'A2M', 'A4GALT', 'A4GNT', 'AAAS']
Using trait column: Retinoblastoma
Shape after handling missing values: (0, 1)
No samples remain after handling missing values. The dataset cannot be processed further.
Abnormality detected in the cohort: GSE68950. Preprocessing failed.
Data quality check failed. The dataset is not suitable for association studies.
